# Acceso a la base de datos y asignacion de inputs y output

In [59]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

vuelos = 'C:\\Users\\max80\\OneDrive\\Documents\\DATATHON 2024\\Filghts TEC_Valid.csv'  #Variable con ruta a base de datos de vuelos
df_vuelos = pd.read_csv(vuelos)  #data frame de la base de datos de vuelos

df_vuelos['STD'] = pd.to_datetime(df_vuelos['STD']) #Datos STD convertidos a fecha
df_vuelos['Año'] = df_vuelos['STD'].dt.year
df_vuelos['Mes'] = df_vuelos['STD'].dt.month
df_vuelos = df_vuelos.sort_values(by = 'STD', ascending = True)

df_vuelos23 = df_vuelos.loc[df_vuelos['Año'] == 2023]  #Filtrado de datos de 2023 (los datos completos)
df_vuelos23 = df_vuelos23.dropna()  #Eliminar lineas con celdas vacias

categorias = df_vuelos23[['Flight_ID','Destination_Type']]  #Categorias tomadas en cuenta para la prediccion
numericas = df_vuelos23[['Capacity','Mes','Bookings']]    #Datos numericos tomados en cuenta para la prediccion

encoder = LabelEncoder()  #Codificador de categorias
categorias = categorias.apply(encoder.fit_transform) #Codificacion

X = pd.concat([pd.DataFrame(numericas), categorias], axis=1) #Inputs array
Y = df_vuelos23['Passengers']  #Output

# Entrenamiento del modelo

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)  #Particion de los datos de output en entrenamiento y prueba

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  #Modelo de red neuronal con 3 capas
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer = 'adam', loss = 'mean_squared_error')   #Compilador del modelo, optimizador y funcion de error
model.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))  #Entrenmiento

Epoch 1/40


C:\Users\max80\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3044/3044 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 2489700.7500 - val_loss: 802.5950
Epoch 2/40
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 701.2096 - val_loss: 1723.0565
Epoch 3/40
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1076.3759 - val_loss: 2584.3364
Epoch 4/40
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1329.1281 - val_loss: 1941.8954
Epoch 5/40
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1347.7972 - val_loss: 762.2828
Epoch 6/40
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1152.6071 - val_loss: 733.7626
Epoch 7/40
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1004.0920 - val_loss: 1097.8861
Epoch 8/40
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1091.7396 - val_loss: 7382.1548
Epoch 9/40
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1055.1178 - val_loss: 5110.3916
Epoch 10/40
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1041.5192 - val_loss: 1887.2521
Epoch 11/40
3044/3044 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1085.597

# Predicciones

In [63]:
df_vuelos24 = df_vuelos.loc[df_vuelos['Año'] == 2024]
categorias_2024 = df_vuelos24[['Flight_ID','Destination_Type']]
numericas_2024 = df_vuelos24[['Capacity','Mes','Bookings']]

encoder = LabelEncoder()
categorias_2024 = categorias_2024.apply(encoder.fit_transform)

X_2024 = pd.concat([pd.DataFrame(numericas_2024), categorias_2024], axis=1)
prediccion = model.predict(X_2024) #Generacion de predicciones usando la red neuronal

3807/3807 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


In [64]:
prediccion = prediccion.flatten()  #Convierte el array multidimensional en unidimensional
df_vuelos24.loc[df_vuelos24['Passengers'].isnull(), 'Passengers'] = prediccion
display(df_vuelos24)

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Año,Mes
4663,5af600e8e3131804971e9764659a4e50,XA-VIY,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2024-01-01 00:05:00,2024-01-01 05:05:00,186,3.35133,NaN,2024,1
54136,aa14abe062912b5497a400a8f3473407,XA-VBT,BM,AO,Ciudad Principal,Ciudad Fronteriza,2024-01-01 00:30:00,2024-01-01 05:25:00,220,3.35133,NaN,2024,1
182265,803b0d7b2c8ae03f9f098eb1686d683a,XA-VXJ,BM,BB,Ecoturismo,Ciudad Fronteriza,2024-01-01 00:55:00,2024-01-01 06:40:00,240,3.35133,NaN,2024,1
9935,479ec62310c8a89c85914816e29e81ec,XA-VAA,BM,AD,MX Amigos y Familia,Ciudad Fronteriza,2024-01-01 01:00:00,2024-01-01 05:50:00,186,3.35133,NaN,2024,1
172070,26a1ddf4c763fc84f7734ef5308cf6de,XA-VAP,BM,BA,Ciudad Principal,Ciudad Fronteriza,2024-01-01 01:00:00,2024-01-01 06:15:00,186,3.35133,NaN,2024,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61970,28023a973f3549f6aa8137a4d77edf56,NaN,AW,AO,Ciudad Principal,Ciudad Principal,2024-12-31 19:25:00,2024-12-31 21:00:00,186,3.35133,NaN,2024,12
86122,1055cf9182d8c679ff176c6ab61c5cc9,NaN,AO,AT,Ciudad Principal,Ciudad Principal,2024-12-31 20:10:00,2024-12-31 21:40:00,180,3.35133,NaN,2024,12
196391,e186466ea508a0b053312c424546f2f4,NaN,AT,BE,Playa,Ciudad Principal,2024-12-31 20:20:00,2024-12-31 21:35:00,220,3.35133,NaN,2024,12
180475,e498ec13ae5b9493354e63f414d684c9,NaN,BT,BA,Ciudad Principal,Playa,2024-12-31 21:05:00,2024-12-31 22:20:00,186,3.35133,NaN,2024,12


In [65]:
prediccion_23 = model.predict(X)
prediccion_23 = prediccion_23.flatten()
df_vuelos23['Neurona'] = prediccion_23
display(df_vuelos23)

3804/3804 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Año,Mes,Neurona
6696,030f35839975b1721e2427db425ef402,XA-VAJ,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-01-02 00:20:00,2023-01-02 05:20:00,186,120.0,68.0,2023,1,136.547958
178553,7b9f51daca0f560676d28aee3d5666f7,XA-VAW,BM,BA,Ciudad Principal,Ciudad Fronteriza,2023-01-02 00:30:00,2023-01-02 06:00:00,186,170.0,99.0,2023,1,168.227005
53258,d4ac6f26a94efea00002147f15d96a41,XA-VIU,BM,AO,Ciudad Principal,Ciudad Fronteriza,2023-01-02 00:35:00,2023-01-02 05:30:00,186,185.0,89.0,2023,1,165.442215
123223,555496d4217768d1cc276ffbe83235d5,XA-VXF,AT,AU,Ecoturismo,Ciudad Principal,2023-01-02 00:55:00,2023-01-02 02:45:00,240,168.0,118.0,2023,1,204.346252
53248,01cb03565042d43a1deafae09eef0ca1,XA-VAN,BM,AO,Ciudad Principal,Ciudad Fronteriza,2023-01-02 01:15:00,2023-01-02 06:10:00,186,165.0,92.0,2023,1,155.945847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221868,a665d50e95bf905d862563facacbf05a,XA-VXA,AO,BM,Ciudad Fronteriza,Ciudad Principal,2023-12-31 23:00:00,2024-01-01 00:05:00,240,201.0,90.0,2023,12,189.749573
92561,6dda5bd834419dd5b900408b0f5fd77b,XA-VAU,BM,AT,Ciudad Principal,Ciudad Fronteriza,2023-12-31 23:05:00,2024-01-01 04:30:00,186,131.0,81.0,2023,12,158.134750
105522,c711d31ee17b65d788d2df7ebde346f1,XA-VBI,AK,AT,Ciudad Principal,Playa,2023-12-31 23:15:00,2024-01-01 00:45:00,220,217.0,126.0,2023,12,213.204407
57082,dc6b4fa09794681fbf19cdd98f5a942e,XA-VIF,BM,AO,Ciudad Principal,Ciudad Fronteriza,2023-12-31 23:45:00,2024-01-01 04:40:00,186,168.0,75.0,2023,12,159.821808
